In [ ]:
!pip install wandb
!pip install monai
!pip install matplotlib
!pip install nibabel
import datetime
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import monai
from PIL import Image
import torch
import nibabel as nib
from monai.transforms import (
    AddChanneld,
    LoadImage,
    LoadImaged,
    Orientationd,
    Rand3DElasticd,
    RandAffined,
    Spacingd,
)
from monai.config import print_config
from monai.apps import download_and_extract
import wandb

In [ ]:

os.environ["WANDB_NOTEBOOK_NAME"] = "./Last_version_Main_script_GAN_def.ipynb"
wandb.login()

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)
data_path = r'/content/drive/My drive/Deep Learning/ACDC/training'


In [ ]:
def get_dict():
  data_path = r'/content/drive/My Drive/Deep Learning/ACDC/training'

  patient_images = glob.glob(os.path.join(data_path,'*'))
  dicts = []
  for images_patients in patient_images:
        # find the binary mask that belongs to the original image, based on indexing in the filename
        mask_path = glob.glob(os.path.join(images_patients,'*gt.nii.gz'))
        data = glob.glob(os.path.join(images_patients,'*'))
        for idx in data:
          image_index = os.path.split(idx)[1].split('_')[-1].split('.')[0]
          patient_number = os.path.split(idx)[1].split('_')
          if image_index != 'gt' and image_index != '4d' and image_index != 'Info':
                 img = os.path.join(f'{images_patients}/{patient_number[0]}_{image_index}.nii.gz')
                 mask = os.path.join(f'{images_patients}/{patient_number[0]}_{image_index}_gt.nii.gz')
                 #time_dim = os.path.join(f'{images_patients}/{patient_number[0]}_4d.nii.gz')
                 if os.path.exists(mask) and os.path.exists(img):
                    dicts.append({'image':img,'mask':mask,'patient_info':f'{patient_number[0]}_{image_index}'})
  return dicts


        
          



In [ ]:
total_list = get_dict()
val_list = total_list[0:20]
train_list = total_list[19:-1]
test_train_list = total_list[0:2]
print(test_train_list)



In [ ]:
 
train_dataset = monai.data.Dataset(train_list)
val_dataset = monai.data.Dataset(val_list)
loader = LoadImaged(keys=("image", "mask"))

data_dict = loader(train_dataset[1])
# print(f"input:, {train_data_dicts[0]}")
print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['mask'].shape}")
print(f"image pixdim:\n{data_dict['image_meta_dict']['pixdim']}")

image, mask = data_dict["image"], data_dict["mask"]
plt.figure("visualize", (8, 4))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :, 5])
plt.subplot(1, 2, 2)
plt.title("mask")
plt.imshow(mask[:, :, 5])
plt.show()








In [ ]:
from matplotlib import transforms
import math



# constructDataset from list of paths + transform
transform = monai.transforms.Compose(
[
    LoadImaged(keys=("image", "mask")),
    monai.transforms.AddChanneld(keys=["image","mask"]),
    monai.transforms.CenterSpatialCropd(keys =["image","mask"], roi_size= [128,128,-1]),
    monai.transforms.Spacingd(keys= ["image","mask"],pixdim=(1,1,1),meta_key_postfix = 'image_meta_dict', mode = ['nearest ','nearest']),
    monai.transforms.AdjustContrastd(keys = ["image"], gamma=(0.5)),
    monai.transforms.HistogramNormalized(keys =["image"],num_bins=256, min=0, max=255),
    monai.transforms.AsDiscreted(keys =["mask"], to_onehot=4),
    monai.transforms.AsChannelFirstd(keys = ["image","mask"], channel_dim=- 1),
    monai.transforms.ToTensord(keys=["image","mask"])
    ])






In [ ]:

val_dataset = monai.data.CacheDataset(val_list, transform=transform)
train_dataset = monai.data.CacheDataset(train_list, transform=transform)
test_train_dataset =monai.data.CacheDataset(test_train_list, transform=transform)












In [ ]:

from monai.utils import first
validation_loader = monai.data.DataLoader(val_dataset,)
train_loader = monai.data.DataLoader(train_dataset)
test_train_loader = monai.data.DataLoader(test_train_dataset,batch_size =1,num_workers=2)

print(first(validation_loader)["patient_info"])


In [ ]:



data = first(train_loader)
mask = np.argmax(data["mask"][0,2,:,:,:],0)


plt.imshow(mask)
plt.show()

plt.imshow(data["image"][0,2,0,:,:])
plt.show()

In [ ]:
def from_compose_to_list(transform_compose):
    """
    Transform an object monai.transforms.Compose in a list fully describing the transform.
    /!\ Random seed is not saved, then reproducibility is not enabled.
    """
    from copy import deepcopy
        
    if not isinstance(transform_compose, monai.transforms.Compose):
        raise TypeError("transform_compose should be a monai.transforms.Compose object.")
    
    output_list = list()
    for transform in transform_compose.transforms:
        kwargs = deepcopy(vars(transform))
        
        # Remove attributes which are not arguments
        args = list(transform.__init__.__code__.co_varnames[1: transform.__init__.__code__.co_argcount])
        for key, obj in vars(transform).items():
            if key not in args:
                del kwargs[key]

        output_list.append({"class": transform.__class__, "kwargs": kwargs})
    return output_list

In [ ]:
torch.cuda.get_device_name(torch.cuda.current_device())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'The used device is {device}')

In [ ]:
def Unet(device):
  model = nn.Sequential(monai.networks.nets.UNet(
      spatial_dims=2,
      in_channels=1,
      out_channels=4,
      channels=(16, 32, 64, 128,256),
      strides=(2, 2,2,2)),torch.nn.Softmax(dim =1)
  ).to(device)
  return model

In [ ]:
loss_function =monai.losses.DiceLoss(softmax=False, batch = True)

In [ ]:
from tqdm import tqdm
import wandb
import torch.nn as nn
model = Unet(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, betas=(0.5, 0.999))


run = wandb.init(
    project='ACDC_segmentation_unet2',
    name='run_at_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
    config={
        'loss function': str(loss_function), 
        'lr': optimizer.param_groups[0]["lr"],
        'transform': from_compose_to_list(transform),
        'batch_size': test_train_loader.batch_size,
    }
)

# Do not hesitate to enrich this list of settings to be able to correctly keep track of your experiments!
# For example you should add information on your model...

run_id = run.id # We remember here the run ID to be able to write the evaluation metrics

def log_to_wandb(epoch, train_loss, val_loss, batch_data, outputs):
    """ Function that logs ongoing training variables to W&B """

    # Create list of images that have segmentation masks for model output and ground truth
    #log_imgs = [wandb.Image(img, masks=wandb_masks(mask_output, mask_gt)) for img, mask_output,
     #           mask_gt in zip(batch_data['image'], outputs, batch_data['mask'])]

    # Send epoch, losses and images to W&B
    wandb.log({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss})
    
# TO BE REMOVED
trigger_times = 0
last_loss =100
patience =4
for epoch in tqdm(range(200)):
    model.train()    
    epoch_loss = 0
    step = 0
    
    for batch_data in train_loader: 
        step += 1
        optimizer.zero_grad()
        outputs = model(batch_data["image"][0].float().to(device))
        loss = loss_function(outputs, batch_data["mask"][0].to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    train_loss = epoch_loss/step
    # validation part
    step = 0
    val_loss = 0
    for batch_data in validation_loader:
        step += 1
        model.eval()
        outputs = model(batch_data['image'][0].float().to(device))
        loss = loss_function(outputs, batch_data['mask'][0].to(device))
        val_loss+= loss.item()
    val_loss = val_loss / step   
    log_to_wandb(epoch, train_loss, val_loss, batch_data, outputs)
    current_loss = val_loss
    if current_loss > last_loss: 
            trigger_times += 1
            print('Trigger Times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                break
  
    else:
      print('trigger times: 0')
      trigger_times = 0
      last_loss = current_loss

# Store the network parameters        
torch.save(model.state_dict(), r'/content/drive/My Drive/Deep Learning/ACDC/trained_Unet_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.pt')
run.finish()

In [ ]:
import numbers
model = Unet(device)
model.load_state_dict(torch.load(r'/content/drive/My Drive/Deep Learning/ACDC/trained_Unet_20220620-073124.pt'))

output = model(first(validation_loader)['image'][0].to(device))
output = np.argmax(output.detach().cpu().numpy(),1)

num = 4
print(output.shape)
plt.figure(figsize=(12.80,7.20))
plt.subplot(2,1,1)
plt.imshow(output[num])
plt.colorbar()
plt.subplot(2,1,2)
plt.imshow(first(validation_loader)['image'][0][num].squeeze(),cmap ='gray')

In [ ]:
import torch.nn as nn
class Discriminator_CNN(nn.Module):
    
    def __init__(self):
        super(Discriminator_CNN, self).__init__()
        self.layers = nn.Sequential(nn.Conv2d(in_channels= 5, out_channels=64, kernel_size=5, stride=2, padding=2),
                                    nn.LeakyReLU(0.2),
                                    nn.Dropout(0.3),
                                    nn.BatchNorm2d(num_features = 64),
                                    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2),
                                    nn.LeakyReLU(0.2),
                                    nn.Dropout(0.3),
                                    nn.BatchNorm2d(num_features = 128),
                                    nn.Flatten(),
                                    nn.Linear(in_features=131072, out_features=1),
                                    nn.Sigmoid())
        
    def forward(self, x):
        return self.layers(x)
  

In [ ]:

def AddNoise(data):
    gaussian = monai.transforms.RandGaussianNoise(prob =1,mean =0.5, std = 0.1)
    softmax = torch.nn.Softmax(dim=1)
    data = gaussian(data)
    data = torch.clamp(data, min= 0,max =1)
    data =softmax(data)
    return data

In [ ]:
def validation(epoch,generator, device, valid_loader, loss_function,train_loss,gen_loss,dis_loss,dis2_loss):

    generator.eval()
    loss_total = 0

    # Test validation data
    with torch.no_grad():
        for data in valid_loader:

            outputs = generator(data['image'][0].float().to(device))
            val_loss = loss_function(outputs, data['mask'][0].to(device))
            loss_total += val_loss.item()
        val_loss = loss_total / len(valid_loader) 
        log_to_wandb(epoch, train_loss, val_loss,data,gen_loss,dis_loss,dis2_loss)

    return val_loss

In [ ]:
data = first(train_loader)
mask_data = data['mask'][0]
data_noise = AddNoise(mask_data)
arg_data = np.argmax(data_noise.detach().cpu().numpy(),1)

plt.subplot(2,1,1)
plt.imshow(data_noise[5,1,:,:])
plt.colorbar()
plt.subplot(2,1,2)
plt.imshow(arg_data[5,:,:])


In [ ]:
from tqdm import tqdm
import wandb
 


def log_to_wandb(epoch, train_loss, val_loss, batch_data,gen_loss,dis_loss,dis2_loss):
    """ Function that logs ongoing training variables to W&B """

    # Create list of images that have segmentation masks for model output and ground truth
    #log_imgs = [wandb.Image(img, masks=wandb_masks(mask_output, mask_gt)) for img, mask_output,
     #           mask_gt in zip(batch_data['image'], outputs, batch_data['mask'])]

    # Send epoch, losses and images to W&B
    wandb.log({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss,'dis_loss': dis_loss,'dis_loss2':dis2_loss,'gen_loss':gen_loss})

def train_gan(epochs,lr_gen,lr_dis,b1,b2,patience,devide_epoch,gen_epoch ,weight_gen,weight_dis,device,train_loader,validation_loader):
  # Get networks
  discriminator = Discriminator_CNN().to(device)
  generator = Unet(device)

  loss_dis= nn.BCELoss()
  loss_dice = monai.losses.DiceLoss(softmax= False , batch=True)
  # Configure optimizers and loss function
  optimizer_dis = torch.optim.Adam(discriminator.parameters(), lr=lr_dis, betas=(b1, b2))
  optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr_gen, betas=(b1, b2))

  run = wandb.init(
    project='ACDC_segmentation_def',
    name='run_at_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
    config={
        'loss function dis': str(loss_dis), 
        'loss function gen': str(loss_dice), 
        'lr_dis': optimizer_dis.param_groups[0]["lr"],
        'lr_gen': optimizer_gen.param_groups[0]["lr"],
        'b1':b1,
        'b2': b2,
        'Weight_gen': weight_gen,
        'Weight_dis': weight_dis,
        'devide_epoch': devide_epoch,
        "generrator epoch":gen_epoch,
        'transform': from_compose_to_list(transform),
        'batch_size': train_loader.batch_size,
    }
)
# Do not hesitate to enrich this list of settings to be able to correctly keep track of your experiments!
# For example you should add information on your model...

  run_id = run.id # We remember here the run ID to be able to write the evaluation metrics

  last_loss = 100
  triggertimes = 0


  for epoch in tqdm(range(1, epochs+1)):
    
    # Wrap dataloader into tqdm such that we can print progress while training

        generator.train()

        epoch_GAN_loss = 0
        epoch_gen_loss = 0
        epoch_dis2_loss = 0
        epoch_dis_loss = 0
        step_dis = 0
        step_gen = 0
        loop_2 = 0
        dis_loss = 0
        for batch_data in train_loader:
            
            loop_2 += 1
            if epoch > gen_epoch and loop_2 % devide_epoch > 0:
              step_dis +=1
              for param in generator.parameters():
                  param.requires_grad = False
              for param in discriminator.parameters():
                  param.requires_grad = True
              discriminator.zero_grad()

               # Generate some fake MNIST images using the generator
              fake_images = generator(batch_data["image"][0].float().to(device))
              input_images = batch_data["image"][0].float().to(device)
              target_images = batch_data["mask"][0].to(device)
              target_images_with_noise = AddNoise(target_images)
            

            # Concatenate the fake and real images
              
              dis_input_1 = torch.cat((fake_images, input_images), dim= 1)
              dis_input_2 = torch.cat((target_images_with_noise, input_images),dim= 1)
              dis_input_def = torch.cat((dis_input_1,dis_input_2),dim=0)


              predictions = discriminator(dis_input_def)
              dis_labels_0 = torch.zeros((batch_data["image"][0].shape[0], 1), device=device)
              dis_labels_1 = torch.ones((batch_data["image"][0].shape[0], 1), device=device)
              dis_labels_def = torch.cat((dis_labels_0,dis_labels_1),dim=0)
              
              
              
              dis_loss_batch = loss_dis(predictions, dis_labels_def)
              
                
              dis_loss_batch.backward()
              optimizer_dis.step()
              epoch_dis_loss += dis_loss_batch
            if epoch < gen_epoch or loop_2 % devide_epoch == 0:
              step_gen +=1
              for param in generator.parameters():
                param.requires_grad = True
              for param in discriminator.parameters():
                param.requires_grad = False
              generator.zero_grad()
              fake_images = generator(batch_data["image"][0].float().to(device))
              input_images = batch_data["image"][0].float().to(device)
              target_images = batch_data["mask"][0].to(device)
              dis_input = torch.cat((fake_images,input_images),dim=1)
              predictions = discriminator(dis_input)
              gen_labels = torch.zeros((batch_data["image"][0].shape[0], 1), device=device)
              dis_loss_2 = loss_dis(predictions,gen_labels)
              gen_loss = loss_dice(target_images,fake_images)
              if epoch < gen_epoch:
                GAN_loss = weight_gen*gen_loss
              else:
                GAN_loss = (weight_gen*gen_loss)+(weight_dis*dis_loss_2)


              GAN_loss.backward()
              optimizer_gen.step()
              epoch_GAN_loss += GAN_loss.item()
              epoch_gen_loss += gen_loss.item()
              epoch_dis2_loss += dis_loss_2.item()

        train_loss = epoch_GAN_loss/step_gen
        gen_loss = epoch_gen_loss/step_gen
        if step_dis > 0:
          dis_loss = epoch_dis_loss/step_dis
        else:
          dis_loss = 0
        dis2_loss = epoch_dis2_loss/step_gen
          

        current_loss = validation(epoch,generator, device, validation_loader, loss_dice,train_loss,gen_loss,dis_loss,dis2_loss)
        print('The Current Loss:', current_loss)

        if current_loss > last_loss and epoch> gen_epoch+10:
            trigger_times += 1
            print('Trigger Times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                break

        else:
            print('trigger times: 0')
            trigger_times = 0
            
            last_loss = current_loss

       
        
              
           

            #torch.autograd.set_detect_anomaly(True)
            
            
            # Train generator with a new batch of generated samples
            
            
            # From the generator's perspective, the discriminator should predict
            # ones for all samples
            
            
            
            
            
            # Train the GAN to predict ones
            
              
        
        

  torch.save(generator.state_dict(), r'/content/drive/My Drive/Deep Learning/ACDC/trainedGAN_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.pt')
  run.finish()
  return generator

In [ ]:
validation_loader = monai.data.DataLoader(val_dataset, batch_size=6)
train_loader = monai.data.DataLoader(train_dataset, batch_size=6)
test_train_loader = monai.data.DataLoader(test_train_dataset, batch_size=2)


In [ ]:
generator = train_gan(epochs = 1000,lr_gen = 0.002,lr_dis = 0.00005,b1 = 0.5,b2 = 0.999,
                      patience =10,devide_epoch =2,gen_epoch =20,weight_gen =5,weight_dis =1,device = device,train_loader = train_loader,validation_loader = validation_loader)

In [ ]:
def Unet(device):
  model = nn.Sequential(monai.networks.nets.UNet(
      spatial_dims=2,
      in_channels=1,
      out_channels=4,
      channels=(16, 32, 64, 128,256),
      strides=(2, 2,2,2)),torch.nn.Softmax(dim=1)
  ).to(device)
  return model


GAN = Unet(device)
GAN.load_state_dict(torch.load(r'/content/drive/My Drive/Deep Learning/ACDC/trainedGAN_20220620-074121.pt'))
GAN.eval()
output_GAN = GAN(first(validation_loader)['image'][0].to(device))
output_GAN = np.argmax(output_GAN.detach().cpu().numpy(),1)


Unet = Unet(device)
Unet.load_state_dict(torch.load(r'/content/drive/My Drive/Deep Learning/ACDC/trained_Unet_20220620-073124.pt'))

output_unet = Unet(first(validation_loader)['image'][0].to(device))
output_unet = np.argmax(output_unet.detach().cpu().numpy(),1)


num = 2
mask_gt = np.argmax(first(validation_loader)['mask'][0,num],0)

plt.figure(figsize=(12.80,7.20))
plt.subplot(2,2,2)
plt.imshow(output_GAN[num])
plt.title('Segmentatie Unet+GAN')
plt.subplot(2,2,1)
plt.imshow(output_unet[num])
plt.title('Segmentatie Unet')
plt.subplot(2,2,3)
plt.imshow(mask_gt)
plt.title('Segmentatie Groundtruth')
plt.subplot(2,2,4)
plt.imshow(first(validation_loader)['image'][0][num].squeeze(),cmap ='gray')
plt.title('MRI Scan')
plt.tight_layout()
plt.savefig(r'/content/drive/My Drive/Deep Learning/ACDC/Table_1.png', dpi = 600)


In [ ]:
def get_dict_test():
  data_path = r'/content/drive/My Drive/Deep Learning/ACDC/testing'

  patient_images = glob.glob(os.path.join(data_path,'*'))
  dicts = []
  for images_patients in patient_images:
        # find the binary mask that belongs to the original image, based on indexing in the filename
        mask_path = glob.glob(os.path.join(images_patients,'*gt.nii.gz'))
        data = glob.glob(os.path.join(images_patients,'*'))
        for idx in data:
          image_index = os.path.split(idx)[1].split('_')[-1].split('.')[0]
          patient_number = os.path.split(idx)[1].split('_')
          if image_index != 'gt' and image_index != '4d' and image_index and 'Info':
                 img = os.path.join(f'{images_patients}/{patient_number[0]}_{image_index}.nii.gz')
                 #time_dim = os.path.join(f'{images_patients}/{patient_number[0]}_4d.nii.gz')
                 if os.path.exists(img):
                    dicts.append({'image':img, 'dict':f'{patient_number[0]}/{patient_number[0]}_{image_index}'})
  return dicts

In [ ]:
test_list = get_dict_test()
print(test_list)
transform_test = monai.transforms.Compose(
[
    LoadImaged(keys=("image")),
    monai.transforms.AddChanneld(keys=["image"]),
    monai.transforms.CenterSpatialCropd(keys =["image"], roi_size= [128,128,-1]),
    monai.transforms.Spacingd(keys= ["image"],pixdim=(1,1,1),meta_key_postfix = 'image_meta_dict', mode =['nearest']),
    monai.transforms.AdjustContrastd(keys = ["image"], gamma=(0.5)),
    monai.transforms.HistogramNormalized(keys =["image"],num_bins=256, min=0, max=255),
    monai.transforms.AsChannelFirstd(keys = ["image"], channel_dim=- 1),
    monai.transforms.ToTensord(keys=["image"])
    ])

test_dataset = monai.data.CacheDataset(test_list, transform=transform_test)
test_loader = monai.data.DataLoader(test_dataset, batch_size =1, num_workers = 2)

trans_inv = monai.transforms.Compose(
[
    monai.transforms.CenterSpatialCropd(keys =["mask"], roi_size= [128,128,-1]),
    monai.transforms.Spacingd(keys= ["image"],pixdim=(1,1,1),meta_key_postfix = 'image_meta_dict', mode =['nearest']),
    monai.transforms.ToTensord(keys=["image"])
    ])





In [ ]:
 t = monai.transforms.ToTensord(keys=["image"])
 print(isinstance(t,monai.transforms.InvertibleTransform))

In [ ]:
def save_nii(img_path, data, affine, header):
    """
    Function to save a 'nii' or 'nii.gz' file.

    Parameters
    ----------

    img_path: string
    Path to save the image should be ending with '.nii' or '.nii.gz'.

    data: np.array
    Numpy array of the image data.

    affine: list of list or np.array
    The affine transformation to save with the image.

    header: nib.Nifti1Header
    The header that define everything about the data
    (pleasecheck nibabel documentation).
    """
    nimg = nib.Nifti1Image(data,affine =affine, header=header)
    nimg.to_filename(img_path)

In [ ]:
def Unet(device):
  model = nn.Sequential(monai.networks.nets.UNet(
      spatial_dims=2,
      in_channels=1,
      out_channels=4,
      channels=(16, 32, 64, 128,256),
      strides=(2, 2,2,2)),torch.nn.Softmax(dim=1)
  ).to(device)
  return model


model = Unet(device)
model.load_state_dict(torch.load(r'/content/drive/My Drive/Deep Learning/ACDC/trainedGAN_20220620-074121.pt'))

path = r'/content/drive/My Drive/Deep Learning/ACDC/results/'
for patient in test_loader:
    output = model(patient["image"][0].float().to(device))
    output = np.argmax(output.detach().cpu().numpy(),1)
    patient_dict = patient["dict"]
    patient_number = os.path.split(patient_dict[0])[1].split('_')[0]
    patient_frame = os.path.split(patient_dict[0])[1].split('_')[-1]

    if patient_frame == 'frame01':
      save_nii(path+'ED/'+str(patient_number)+'_ED.nii.gz',output,None,None)
    else:
      save_nii(path+'ES/'+str(patient_number)+'_ES.nii.gz',output,None,None)

    
    
  






In [ ]:
def load_nii(img_path):
    """
    Function to load a 'nii' or 'nii.gz' file, The function returns
    everyting needed to save another 'nii' or 'nii.gz'
    in the same dimensional space, i.e. the affine matrix and the header

    Parameters
    ----------

    img_path: string
    String with the path of the 'nii' or 'nii.gz' image file name.

    Returns
    -------
    Three element, the first is a numpy array of the image values,
    the second is the affine transformation of the image, and the
    last one is the header of the image.
    """
    nimg = nib.load(img_path)
    return nimg.get_data(), nimg.affine, nimg.header

In [ ]:
data, affine, header =load_nii(r'/content/drive/My Drive/Deep Learning/ACDC/results/ED/patient150_ED.nii.gz')
data = trans_inv.inverse(data)
plt.imshow(data[5])

In [ ]:

import pandas as pd
def Unet(device):
  model = nn.Sequential(monai.networks.nets.UNet(
      spatial_dims=2,
      in_channels=1,
      out_channels=4,
      channels=(16, 32, 64, 128,256),
      strides=(2, 2,2,2)),torch.nn.Softmax(dim=1)
  ).to(device)
  return model

loss_dice = monai.losses.DiceLoss(softmax= False , batch=True)
model = Unet(device)
model.load_state_dict(torch.load(r'/content/drive/My Drive/Deep Learning/ACDC/trained_Unet_20220620-073124.pt'))
patient_list = []
path = r'/content/drive/My Drive/Deep Learning/ACDC/results/'
count = 0
for patient in validation_loader:
    count+=1
    output = np.argmax(model(patient["image"][0].float().to(device)).detach().cpu().numpy(),1)
    ground_truth = np.argmax(patient["mask"][0].detach().cpu().numpy(),1)
    patient_info = patient['patient_info']
    patient_number = os.path.split(patient_info[0])[1].split('_')[0]
    patient_frame = os.path.split(patient_info[0])[1].split('_')[-1]
    print(patient_frame)
    if patient_frame == 'frame01':
      patient_info = str(patient_number)+'_ED'
    else:
      patient_info = str(patient_number)+'_ES'
    res = metrics(ground_truth,output,[1,1,1])
    res = np.reshape(res,(1,9))
    if count == 1:
      pd1 = pd.DataFrame(data=res,columns = ("Dice LV", "Volume LV", "Err LV(ml)",
    "Dice RV","Volume RV"," Err RV(ml)", "Dice MYO", "Volume MYO", "Err MYO(ml)"),index = [patient_info])
    else:
      pd2 = pd.DataFrame(data=res,columns =("Dice LV", "Volume LV", "Err LV(ml)",
    "Dice RV","Volume RV"," Err RV(ml)", "Dice MYO", "Volume MYO", "Err MYO(ml)"), index = [patient_info])
      pd1 = pd1.append(pd2)
      


writer = pd.ExcelWriter(r'/content/drive/My Drive/Deep Learning/ACDC/data.xlsx',egine='xlsxwriter') 
pd1.to_excel(writer,header =True)
writer.save()


In [ ]:
def metrics(img_gt, img_pred, voxel_size):
    """
    Function to compute the metrics between two segmentation maps given as input.

    Parameters
    ----------
    img_gt: np.array
    Array of the ground truth segmentation map.

    img_pred: np.array
    Array of the predicted segmentation map.

    voxel_size: list, tuple or np.array
    The size of a voxel of the images used to compute the volumes.

    Return
    ------
    A list of metrics in this order, [Dice LV, Volume LV, Err LV(ml),
    Dice RV, Volume RV, Err RV(ml), Dice MYO, Volume MYO, Err MYO(ml)]
    """

    if img_gt.ndim != img_pred.ndim:
        raise ValueError("The arrays 'img_gt' and 'img_pred' should have the "
                         "same dimension, {} against {}".format(img_gt.ndim,
                                                                img_pred.ndim))

    res = []
    # Loop on each classes of the input images
    for c in [3, 1, 2]:
        # Copy the gt image to not alterate the input
        gt_c_i = np.copy(img_gt)
        gt_c_i[gt_c_i != c] = 0

        # Copy the pred image to not alterate the input
        pred_c_i = np.copy(img_pred)
        pred_c_i[pred_c_i != c] = 0

        # Clip the value to compute the volumes
        gt_c_i = np.clip(gt_c_i, 0, 1)
        pred_c_i = np.clip(pred_c_i, 0, 1)

        # Compute the Dice
        dice = 1-loss_dice(torch.tensor(gt_c_i), torch.tensor(pred_c_i)).detach()

        # Compute volume
        volpred = pred_c_i.sum() * np.prod(voxel_size) / 1000.
        volgt = gt_c_i.sum() * np.prod(voxel_size) / 1000.

        res += [dice, volpred, volpred-volgt]

    return res

In [ ]:


print(pd1)


